In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
pd.set_option('display.max_columns', None)

In [2]:
# geostl nghbd codes:
    # gravois park = 19
    # benton park west = 30
    # dutchtown = 16
# https://dynamic.stlouis-mo.gov/citydata/newdesign/sqlsearch.cfm
gravois_total_parcels = 1733
benton_park_west_total_parcels = 1693
dutchtown_total_parcels = 4850

In [3]:
csv_folder = 'stl_vacancy_data/'
path = "stl_vacancy_data/*.csv"
csv_list = []

bpw_gp_vacancy_cat_df = pd.DataFrame(columns=['Date', 'V_Indeterminate', 'V_Possible', 'V_Very_likely', 'V_Definite'])
bpw_gp_burden_cat_df = pd.DataFrame(columns=['Date', 'B_Zero', 'B_Low', 'B_Medium', 'B_High'])

dutchtown_vacancy_cat_df = pd.DataFrame(columns=['Date', 'V_Indeterminate', 'V_Possible', 'V_Very_likely', 'V_Definite'])
dutchtown_burden_cat_df = pd.DataFrame(columns=['Date', 'B_Zero', 'B_Low', 'B_Medium', 'B_High'])

for fname in glob.glob(path):
    csv_name = re.findall(r'stl_vacancy_data_\d\d\d\d-\d\d-\d\d.csv', fname)[0]
    csv_list.append(csv_name)

In [4]:
class VacancyTransformer():
    def __init__(self, csv_name):
        self.date = re.findall(r'\d\d\d\d-\d\d-\d\d', csv_name)[0]
        self.csv_name = csv_name
        
    def load_raw_df(self):
        self.raw_df = pd.read_csv(csv_folder+self.csv_name)
        return self.raw_df

    def create_regional_df(self, region):

        valid = {'bpw-gp', 'dutchtown'}
        if region not in valid:
            raise ValueError("results: status must be one of %r." % valid)

        if region == 'bpw-gp':
            region_df = self.raw_df[(self.raw_df['NhdName'] == 'Benton Park West') | (self.raw_df['NhdName'] == 'Gravois Park')]

        if region == 'dutchtown':
            region_df = self.raw_df[self.raw_df['NhdName'] == 'Dutchtown']
        
        return region_df
        
    def calc_vacancy_cats(self, df, aggregate_df):
        counts = df['VacancyCat'].value_counts()
        data = [self.date] + counts[['Indeterminant', 'Possible', 'Very Likely', 'Definite']].tolist()
        
        aggregate_df.loc[len(aggregate_df.index)] = data

    def calc_burden_cats(self, df, aggregate_df):
        counts = df['BurdenCat'].value_counts()
        zero_cat = counts['Zero']
        low_cat = sum(counts[['Minimal', 'Very Low', 'Low']])
        med_cat = sum(counts[['Medium Low', 'Medium', 'Medium High', 'Somewhat High']])
        high_cat = sum(counts[['High', 'Very High', 'Extremely High']])
        
        aggregate_df.loc[len(aggregate_df.index)] = [self.date, zero_cat, low_cat, med_cat, high_cat]

    def calc_groupby_counts(self, df, cat_list, region):
        
        valid = {'stl','bpw-gp', 'dutchtown'}
        if region not in valid:
            raise ValueError("results: status must be one of %r." % valid)
        
        for cat in cat_list:
            
            # VacancyCat:
            vac_df = df.groupby(cat)['VacancyCat'].value_counts().to_frame().unstack()
            vac_df.columns = vac_df.columns.droplevel()
            vac_df.fillna(value=0, inplace=True)
            vac_value_name = vac_df.columns.name
            vac_index_name = vac_df.index.name
            vac_df = vac_df[['Indeterminant', 'Possible', 'Very Likely', 'Definite']]
            vac_df.rename(columns={'Indeterminant': 'V_Indeterminant',
                               'Possible': 'V_Possible',
                               'Very Likely': 'V_Very_Likely',
                               'Definite': 'V_Definite'},
                          inplace=True)
            vac_df.reset_index(inplace=True)
            vac_df.insert(loc=0, column='Date', value=self.date)
            vac_df.to_csv(f'data/temp/type_vacancy_data/{region}/{region}_{vac_index_name}_{vac_value_name}_{self.date}.csv')

            # BurdenCat
            bur_df =  df.groupby(cat)['BurdenCat'].value_counts().to_frame().unstack()
            bur_df.columns = bur_df.columns.droplevel()
            bur_df.fillna(value=0, inplace=True)
            bur_index_name = bur_df.index.name
            bur_value_name = bur_df.columns.name
            bur_df['B_Zero'] = bur_df['Zero']
            bur_df['B_Low'] = bur_df['Minimal'] + bur_df['Very Low'] + bur_df['Low']
            bur_df['B_Medium'] = bur_df['Medium Low'] + bur_df['Medium'] + bur_df['Medium High'] + bur_df['Somewhat High']
            bur_df['B_High'] = bur_df['High'] + bur_df['Very High'] + bur_df['Extremely High']
            bur_df.reset_index(inplace=True)
            bur_df.insert(loc=0, column='Date', value=self.date)
            bur_df = bur_df[['Date', 'Type', 'B_Zero', 'B_Low', 'B_Medium', 'B_High']]
            bur_df.to_csv(f'data/temp/type_burden_data/{region}/{region}_{bur_index_name}_{bur_value_name}_{self.date}.csv')

In [5]:
for csv in csv_list:
    d = VacancyTransformer(csv)
    d.load_raw_df()
    
    bpw_gp_df = d.create_regional_df(region = 'bpw-gp')
    d.calc_vacancy_cats(df=bpw_gp_df, aggregate_df = bpw_gp_vacancy_cat_df)
    d.calc_burden_cats(df=bpw_gp_df, aggregate_df = bpw_gp_burden_cat_df)
    d.calc_groupby_counts(df=bpw_gp_df, cat_list=['Type'], region='bpw-gp')

    dutchtown_df = d.create_regional_df(region = 'dutchtown')
    d.calc_vacancy_cats(df=dutchtown_df, aggregate_df = dutchtown_vacancy_cat_df)
    d.calc_burden_cats(df=dutchtown_df, aggregate_df = dutchtown_burden_cat_df)
    d.calc_groupby_counts(df=dutchtown_df, cat_list=['Type'], region='dutchtown')
    

In [6]:
bpw_gp_burden_cat_df.to_csv('data/temp/bpw-gp_burden.csv')
bpw_gp_vacancy_cat_df.to_csv('data/temp/bpw-gp_vacancy.csv')

dutchtown_burden_cat_df.to_csv('data/temp/dutchtown_burden.csv')
dutchtown_vacancy_cat_df.to_csv('data/temp/dutchtown_vacancy.csv')

In [7]:
path = "data/temp/type_burden_data/bpw-gp/*.csv"
burden_list = []
for fname in glob.glob(path):
    df = pd.read_csv(fname, index_col=0)
    burden_list.append(df)
bpw_gp_type_burden_all = pd.concat(burden_list, ignore_index=True)
bpw_gp_type_burden_all.to_csv('data/bpw-gp_type_burden_all.csv')

In [8]:
path = "data/temp/type_burden_data/dutchtown/*.csv"
burden_list = []
for fname in glob.glob(path):
    df = pd.read_csv(fname, index_col=0)
    burden_list.append(df)
dutchtown_type_burden_all = pd.concat(burden_list, ignore_index=True)
dutchtown_type_burden_all.to_csv('data/dutchtown_type_burden_all.csv')

In [9]:
# path = "data/temp/type_burden_data/Gravois-Jefferson/*.csv"
# burden_list = []
# for fname in glob.glob(path):
#     df = pd.read_csv(fname, index_col=0)
#     burden_list.append(df)
# grav_jeff_type_burden_all = pd.concat(burden_list, ignore_index=True)
# grav_jeff_type_burden_all.to_csv('data/grav_jeff_type_burden_all.csv')

In [10]:
path = "data/temp/type_vacancy_data/bpw-gp/*.csv"
vacancy_list = []
for fname in glob.glob(path):
    df = pd.read_csv(fname, index_col=0)
    vacancy_list.append(df)
bpw_gp_type_vacancy_all = pd.concat(vacancy_list, ignore_index=True)
bpw_gp_type_vacancy_all.to_csv('data/bpw-gp_type_vacancy_all.csv')

In [11]:
path = "data/temp/type_vacancy_data/dutchtown/*.csv"
vacancy_list = []
for fname in glob.glob(path):
    df = pd.read_csv(fname, index_col=0)
    vacancy_list.append(df)
dutchtown_type_vacancy_all = pd.concat(vacancy_list, ignore_index=True)
dutchtown_type_vacancy_all.to_csv('data/dutchtown_type_vacancy_all.csv')

In [12]:
stl_vacancy_cat_df = pd.DataFrame(columns=['Date', 'V_Indeterminate', 'V_Possible', 'V_Very_likely', 'V_Definite'])
stl_burden_cat_df = pd.DataFrame(columns=['Date', 'B_Zero', 'B_Low', 'B_Medium', 'B_High'])

### repeat above process for all STL properties instead of regional

In [13]:
# repeat above process for all STL properties
for csv in csv_list:
    d = VacancyTransformer(csv)
    stl_df = d.load_raw_df()
    
    d.calc_vacancy_cats(df=stl_df, aggregate_df=stl_vacancy_cat_df)
    d.calc_burden_cats(df=stl_df, aggregate_df=stl_vacancy_cat_df)
    d.calc_groupby_counts(df=stl_df, cat_list=['Type'], region='stl')

In [14]:
stl_vacancy_cat_df.to_csv('data/temp/stl_vacancy.csv')
stl_burden_cat_df.to_csv('data/temp/stl_burden.csv')

In [15]:
path = "data/temp/type_burden_data/stl/*.csv"
burden_list = []
for fname in glob.glob(path):
    df = pd.read_csv(fname, index_col=0)
    burden_list.append(df)
stl_type_burden_all = pd.concat(burden_list, ignore_index=True)
stl_type_burden_all.to_csv('data/stl_type_burden_all.csv')

In [16]:
path = "data/temp/type_vacancy_data/stl/*.csv"
vacancy_list = []
for fname in glob.glob(path):
    df = pd.read_csv(fname, index_col=0)
    vacancy_list.append(df)
stl_type_vacancy_all = pd.concat(vacancy_list, ignore_index=True)
stl_type_vacancy_all.to_csv('data/stl_type_vacancy_all.csv')

### get building type data and separate for commercial/residential and LRA/non-LRA

In [30]:
stl_type_vacancy_all = pd.read_csv('data/stl_type_vacancy_all.csv', index_col=0)
bpw_gp_type_vacancy_all = pd.read_csv('data/bpw-gp_type_vacancy_all.csv', index_col=0)
dutchtown_type_vacancy_all = pd.read_csv('data/dutchtown_type_vacancy_all.csv', index_col=0)

In [31]:
stl_type_vacancy_all['Type'].value_counts()

Type
Commercial       27
Duplex           27
Empty Lot        27
Multi-Unit       27
Other            27
Single-Family    27
Mixed-Use        26
Name: count, dtype: int64

In [32]:
stl_type_vacancy_all

,Date,Type,V_Indeterminant,V_Possible,V_Very_Likely,V_Definite
0,2021-03-01,Commercial,254.0,136.0,398.0,225.0
1,2021-03-01,Duplex,364.0,141.0,833.0,1365.0
2,2021-03-01,Empty Lot,578.0,217.0,4349.0,5706.0
3,2021-03-01,Multi-Unit,91.0,48.0,202.0,393.0
4,2021-03-01,Other,170.0,62.0,246.0,248.0
...,...,...,...,...,...,...
183,2023-11-01,Empty Lot,381.0,65.0,6706.0,8756.0
184,2023-11-01,Mixed-Use,20.0,2.0,43.0,63.0
185,2023-11-01,Multi-Unit,84.0,19.0,169.0,496.0
186,2023-11-01,Other,0.0,1.0,3.0,0.0


In [33]:
curr_stl = stl_type_vacancy_all[stl_type_vacancy_all['Date'] == '2023-11-01']
curr_stl

,Date,Type,V_Indeterminant,V_Possible,V_Very_Likely,V_Definite
181,2023-11-01,Commercial,126.0,29.0,403.0,156.0
182,2023-11-01,Duplex,233.0,27.0,525.0,1573.0
183,2023-11-01,Empty Lot,381.0,65.0,6706.0,8756.0
184,2023-11-01,Mixed-Use,20.0,2.0,43.0,63.0
185,2023-11-01,Multi-Unit,84.0,19.0,169.0,496.0
186,2023-11-01,Other,0.0,1.0,3.0,0.0
187,2023-11-01,Single-Family,736.0,105.0,1545.0,3167.0


In [34]:
curr_stl.sum()

Date               2023-11-012023-11-012023-11-012023-11-012023-1...
Type               CommercialDuplexEmpty LotMixed-UseMulti-UnitOt...
V_Indeterminant                                               1580.0
V_Possible                                                     248.0
V_Very_Likely                                                 9394.0
V_Definite                                                   14211.0
dtype: object

In [35]:
parcel_total = 23853


In [36]:
curr_stl = curr_stl.drop('Date', axis=1)

In [37]:
curr_stl.reset_index(inplace=True, drop=True)

In [38]:
curr_stl

,Type,V_Indeterminant,V_Possible,V_Very_Likely,V_Definite
0,Commercial,126.0,29.0,403.0,156.0
1,Duplex,233.0,27.0,525.0,1573.0
2,Empty Lot,381.0,65.0,6706.0,8756.0
3,Mixed-Use,20.0,2.0,43.0,63.0
4,Multi-Unit,84.0,19.0,169.0,496.0
5,Other,0.0,1.0,3.0,0.0
6,Single-Family,736.0,105.0,1545.0,3167.0


In [39]:
com_df = curr_stl.iloc[0]
res_df = curr_stl.iloc[[1, 4, 6]]


In [40]:
res_df

,Type,V_Indeterminant,V_Possible,V_Very_Likely,V_Definite
1,Duplex,233.0,27.0,525.0,1573.0
4,Multi-Unit,84.0,19.0,169.0,496.0
6,Single-Family,736.0,105.0,1545.0,3167.0
